In [ ]:
# pip install geopandas, geopy, opencage, ipywidgets (for tqdm)

In [1]:
import os
# Creating paths
in_folderpath = os.path.join("..", "in")

os.makedirs(in_folderpath, exist_ok=True)

In [2]:
import geopandas as gpd

# Loading the data into A geopandas dataframe
gdf = gpd.read_file(os.path.join(in_folderpath, "datacenters.geojson"))

# Filtering the dataframe to only include locations within the US
gdf_usa = gdf[gdf['country'] == 'USA']

print(len(gdf_usa))

2526


## Correct way of using the geolocator

In [3]:

from opencage.geocoder import OpenCageGeocode
from shapely.geometry import Point
import json
from tqdm import tqdm

api_key = '4176360a8df2453abab8cb3341a7be21'

geolocator = OpenCageGeocode(api_key)

# Iterate through each row to check and update geometry
for i, row in tqdm(gdf_usa.iterrows(), desc="Geocoding Addresses"):

    if row['geometry'] is None or row['geometry'].is_empty:
        
        # Creates an address for the geolocator
        address = f"{row['address']}, {row['postal']}, {row['city']}, {row['state']}, {row['country']}"
        
        # Returns a list of dictionaries with spatial information about the data center 
        location = geolocator.geocode(address)
        
        if location:

            # Retrieves the geometry from the list of dictionaries
            geometry = location[0]['geometry']

            # Adds the retrived latitude and longditude to each row in data frame
            gdf_usa.at[i, 'geometry'] = Point(geometry['lng'], geometry['lat'])
            gdf_usa.at[i, 'latitude'] = geometry['lat']
            gdf_usa.at[i, 'longitude'] = geometry['lng']
        else:
            print("Geometry not found")


gdf_usa_filtered = gdf_usa[gdf_usa['geometry'].notna()]

print(len(gdf_usa_filtered))

# Convert GeoDataFrame to JSON and save
json_str = gdf_usa_filtered.to_json()

json_dict = json.loads(json_str)

with open(os.path.join(in_folderpath, "datacenters_usa.geojson"), 'w') as f:
    json.dump(json_dict, f, indent=2)

print("GeoJSON file has been formatted and saved.")

Geocoding Addresses: 2526it [03:03, 13.76it/s]


2526
GeoJSON file has been formatted and saved.


## Wrong way of using the geolocator

In [ ]:
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import json
from tqdm import tqdm


# Iterate through each row to check and update geometry
for i, row in tqdm(gdf_usa.iterrows()):

    if row['geometry'] is None or row['geometry'].is_empty:
        
        # Dynamically create a geolocator object with a unique user_agent for each row
        geolocator = Nominatim(user_agent=f"geocoding_address_{i}")
        
        # Creates an address for the geolocator
        address = f"{row['address']}, {row['postal']}, {row['city']}, {row['state']}, {row['country']}"
        location = geolocator.geocode(address)
        
        if location:
            # Primary location found, updating the geometry
            gdf_usa.at[i, 'geometry'] = Point(location.longitude, location.latitude)
            gdf_usa.at[i, 'latitude'] = location.latitude
            gdf_usa.at[i, 'longitude'] = location.longitude
        else:
            # Primary location not found, trying alternate address
            alt_address = f"{row['postal']}, {row['city']}, {row['state']}, {row['country']}"
            alt_location = geolocator.geocode(alt_address)

            if alt_location:
                # Alternate location found, updating the geometry
                gdf_usa.at[i, 'geometry'] = Point(alt_location.longitude, alt_location.latitude)
                gdf_usa.at[i, 'latitude'] = alt_location.latitude
                gdf_usa.at[i, 'longitude'] = alt_location.longitude
            else:
                # Neither primary nor alternate location found
                print("Geometry not found")


gdf_usa_filtered = gdf_usa[gdf_usa['geometry'].notna()]

# Convert GeoDataFrame to JSON and save
json_str = gdf_usa_filtered.to_json()

json_dict = json.loads(json_str)

with open(os.path.join(in_folderpath, "datacenters_usa.geojson"), 'w') as f:
    json.dump(json_dict, f, indent=2)

print("GeoJSON file has been formatted and saved.")

## Finding missing temperatures

In [ ]:

import pandas as pd

state_temp = pd.read_csv(os.path.join(in_folderpath, "state_temp.csv"), skiprows=4)

state_temp.head()

In [ ]:
state_temp.rename(columns={'Name' : 'State_Name', 'Value' : 'temperature'}, inplace=True)

state_temp = state_temp[['State_Name','temperature']]

state_temp.head()

In [ ]:

# Calculates the average of the annual high temperature and annual low temperature for Honolulu, Hawaii
avg_temp_hawaii = (84+71)/2

# Creates a data frame with the missing states and their average tempeatures 
missing_states = pd.DataFrame({
    'State_Name': ['Alaska', 'Washington, D.C', 'Hawaii'],
    'temperature': [28.4, 61.2, avg_temp_hawaii]
})

# Creates a boolean condition that checks if the State_Name column contains any of the missing states
states_found = state_temp['State_Name'].isin(['Alaska', 'Washington, D.C', 'Hawaii'])

# If none of the missing states are present in the column, the missing_states data frame and state_temp data frame are merged together  
if not states_found.any():

    state_temp = pd.concat([state_temp, missing_states])

# Sorts the data frame by state name, drops the original index and resets the new one from zero
state_temp = state_temp.sort_values(by='State_Name').reset_index(drop=True)


In [ ]:
state_temp

## References 

### Nominatim

https://medium.com/@gopesh3652/geocoding-with-python-using-nominatim-a-beginners-guide-220b250ca48d 

https://geopy.readthedocs.io/en/stable/ 


### Temperatures

Alaska:
https://www.ncei.noaa.gov/access/monitoring/monthly-report/national/202313 

Hawaii:
https://www.usclimatedata.com/climate/honolulu/hawaii/united-states/ushi0026

Washington DC:
https://www.weather.gov/media/lwx/climate/dcatemps.pdf 
